In [1]:
from wayne_utils import load_data, save_data
import os
import sys
import pandas as pd
from tqdm import tqdm

_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/TKGT"
_OUTPUT_PATH = os.path.join( _ROOT_PATH, "test/LiveSum/v1")

test_path = os.path.join( _ROOT_PATH, "data/LiveSum/test.json")
train_path = os.path.join( _ROOT_PATH, "data/LiveSum/train.json")

test_list = load_data( test_path, "json")           # 754
train_list = load_data( train_path, "json")         # 3017

# Pure-Rule预测

In [2]:
## Rule
from tools.live_sum_rule import main_rule
main_rule( test_list, _OUTPUT_PATH)

Processing texts:   0%|          | 0/754 [00:00<?, ?file/s]

/data/jiangpeiwen2/TKGT/test/LiveSum/tools/utils.py:98: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data[ list_[i]][0] = points_count['away'][i]
/data/jiangpeiwen2/TKGT/test/LiveSum/tools/utils.py:99: FutureWarning: ChainedAssignmentError:

# Rule+llm

In [2]:
from tools.live_sum_rag import get_prompt, post_process
from llm_inferencer import Register, Inferencer
save_path = os.path.join( _ROOT_PATH, "test/LiveSum/v2")
''''''
get_prompt( test_list, save_path )                          # 准备提示词
reg = Register()
reg.list_models()

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 00:15:24,340	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ERROR 12-10 00:15:25 pynccl.py:53] Failed to load NCCL library from libnccl.so.2 .It is expected if you are not running on NVIDIA/AMD GPUs.Otherwise please set the environment variable VLLM_NCCL_SO_PATH to point to the correct nccl library path.
INFO 12-10 00:15:25 pynccl_utils.py:17] Failed to import NCCL library: libnccl.so.2: cannot open shared object file: No such file or directory
INFO 12-10 00:15:25 pynccl_utils.py:18] It is expected if you are not running on NVIDIA GPUs.


Processing texts: 100%|██████████| 754/754 [00:00<00:00, 2325.84file/s]


{'engine_config': {'vllm': {'best_of': 2, 'dtype': 'float16', 'gpu_memory_utilization': 0.9, 'max_model_len': 4096, 'max_tokens': 2048, 'seed': 32, 'stop': ['<|im_end|>', '<|endoftext|>', '<|im_start|>'], 'temperature': 0, 'top_p': 1, 'trust_remote_code': True, 'use_beam_search': True}}, 'local': {'Baichuan2-13B-Chat': '/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Baichuan2-13B/Baichuan2-13B-Chat', 'Baichuan2-7B-Chat': '/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Baichuan2-7B-Chat', 'ChatGLM3-6B': '/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/ChatGLM3-6B/ZhipuAI/chatglm3-6b', 'ChatGLM3-6B-livesum-v1-20epoch': '/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/0', 'ChatGLM3-6B-v1-20epoch': '/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/CPL/v1/models/1', 'ChatGLM3-6B-v1-3epoch': '/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/CPL/v1/models/0', 'ChatGLM3-6B-v1-4epoch': '/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/CPL/v1/models/2', 'Chinese-Mistral-7B-Instruct-v0.1': '/home/jiangpeiw

In [3]:
aaa = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v2/prompt_list_only.pickle", "pickle")

In [6]:
len(aaa[0][0])

410

In [9]:
reg.add_local_model( "Qwen1.5-7B-Chat-livesum-30epoch", "/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4")

In [3]:
kwargs = {}
prompt_list_from_path = os.path.join( save_path, "prompt_list_only.pickle")
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "reader"
kwargs["model_name"] = "Qwen1.5-7B-Chat-livesum-30epoch"
kwargs["gpu_list"] = "0,1,2,3,4,5"
kwargs["local_engine"] = "vllm"
if prompt_list_from_path != None:
    kwargs["prompt_list_from_path"] = prompt_list_from_path
    kwargs["predict_list_to_path"] = prompt_list_from_path.replace( "prompt_list_only.pickle", "predict_lists_rule.pickle")
    kwargs["sample_little"] = None
inferencer = Inferencer( kwargs )

Enginevllm exists, run with vllm


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)


...

WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_re

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
WARNING 12-10 00:15:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)
INFO 12-10 00:15:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v1/models/4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_re

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:28 selector.py:16] Using FlashAttention backend.
INFO 12-10 00:15:38 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:39 gpu_executor.py:94] # GPU blocks: 687, # CPU blocks: 512
INFO 12-10 00:15:40 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:40 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:40 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:40 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:40 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 12-10 00:15:41 gpu_executor.py:94] # GPU blocks:

In [4]:
# 后处理
save_path = "/home/jiangpeiwen2/jiangpeiwen2/TKGT/test/LiveSum/v2"
test_data_path = "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/LiveSum/test.json"
post_process( save_path, test_data_path)

/data/jiangpeiwen2/TKGT/test/LiveSum/tools/utils.py:108: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data[ list_[i]][0] = points_count['away'][i]
/data/jiangpeiwen2/TKGT/test/LiveSum/tools/utils.py:109: FutureWarning: ChainedAssignmentErro